In [9]:
%load_ext autoreload
%autoreload 2

INF = 1e30

import os
from os import path


# set the repository to the git repository
cwd = os.getcwd().split(os.path.sep)
while cwd[-1] != "AttentionGeometry":
    os.chdir("..")
    cwd = os.getcwd().split(os.path.sep)
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from tqdm.notebook import tqdm

# from the environment
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from src.bert_model import BertNliRegu
from src.bert_data_module.esnli import ESNLIDataModule

# specific file for this notebook
from notebooks.BERT.utils.sim_mesure import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ESNLI

In [10]:
muls = ["mul=0.0", "mul=0.5", "mul=0.005", "mul=0.0005"]

In [11]:
data_dir = os.path.join(".cache_bert", "datasets", "EsnliDataSet")

dm = ESNLIDataModule(cache=data_dir,
                   batch_size = 4,
                   num_workers = 4,
                   nb_data = 999 # multiple of three for the consistency
                   )

dm.prepare_data()

dm.setup(stage="test")

test_dataset = dm.test_set
test_dataloader = dm.test_dataloader()

In [12]:
key_evol = np.zeros((12, 4))
val_evol = np.zeros((12, 4))

for i, mul in enumerate(muls):
    ckp = path.join(".cache_bert", "logs", "snli_igrida_trained", "regu_study","layer_4_10", mul, "checkpoints", "best.ckpt")
    model = BertNliRegu.load_from_checkpoint(ckp)
    model = model.to(DEVICE)
    model = model.eval()

    key_map, val_map = main(model, dm)
    key_map = key_map.mean(axis=-1)
    val_map = val_map.mean(axis=-1)

    key_evol[:, i] = key_map
    val_evol[:, i] = val_map

    model = model.cpu()
    del model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


>> The main function for S01EP02 session
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1212, -0.0638,  0.2946,  ...,  0.1812, -0.1859,  0.0445],
         [-0.4105,  0.2600,  0.0346,  ..., -0.0387,  0.5877,  0.6045],
         [-1.0017,  0.1972,  0.2520,  ..., -0.0110,  0.5549,  0.4392],
         ...,
         [-0.1813, -0.0918,  0.0637,  ...,  0.1855,  0.2996,  0.3669],
         [-0.0760, -0.1506,  0.1091,  ...,  0.1385,  0.1243,  0.3614],
         [-0.1337, -0.0533,  0.0068,  ...,  0.2042,  0.1971,  0.3720]],

        [[-0.1212, -0.0638,  0.2946,  ...,  0.1812, -0.1859,  0.0445],
         [-0.4105,  0.2600,  0.0346,  ..., -0.0387,  0.5877,  0.6045],
         [-0.3285, -0.3742, -0.0676,  ...,  0.0160,  0.4278,  0.6640],
         ...,
         [-0.3234, -0.2151,  0.0748,  ...,  0.1398,  0.2333,  0.5553],
         [-0.2031, -0.1439,  0.0809,  ...,  0.1724,  0.2257,  0.4934],
         [-0.2432, -0.1940,  0.0885,  ...,  0.1966,  0.1751,  0.4654]],

        [[-0

ValueError: too many values to unpack (expected 2)